In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
np.random.seed(42)
n_p = 28 # the period
fcst = np.random.poisson(lam=55, size=n_p) 
truth= np.random.poisson(lam=50, size=n_p)

In [2]:
fcst = pd.read_pickle("glgbm_lvls1_12.pkl")
truth= pd.read_pickle("sarima_lvls1_12.pkl")

## Initialization:

In [3]:
# Fixed params:
lead_time = 3
h = 1
b = 19
a = 0.90

# initial position
invt = 0 # initial inventory
wig  = 0 # work-in-progress 
net_invt = 0 # initial net inventory

# list for records:
ch_list = [] # recording costs for holding
cb_list = [] # recording costs for backloging
suc_serv= [] # whether or not there is sucess service (namely, have enoughe inventory to order)

## Forecasted Demand Func.

In [4]:
import pandas as pd
def ob_D_tL(
    t, # current period
    fcst_1, # forecasted demand from models
    L:int = 3 # 
):
    return sum(fcst_1[t:t+L])

## Safety Stock Func.

In [5]:
def ob_ss_t(
    fcst_1_errors,
    alpha,
    L:int = 3 
):
    import numpy as np
    from scipy.stats import norm
    return norm.ppf(alpha)*np.std(fcst_1_errors)*np.sqrt(L)

## Inventory Position Func.

In [6]:
def ob_ip_t(
    ip_t, 
    net_t, 
    w_t, # 
    o_t, # amount of orders at time stamp t
    truth_1, # true values for a single time series
    fcst_1, # forecats for a single time series
    t , # time stamp
    o_t_L = np.nan,
    L:int=3
):
    if t < 3:
        net_t += np.mean(fcst_1[:L]) - truth_1[t] # at time stemp t+1
        w_t   += o_t - np.mean(fcst_1[:L]) # at time stemp t+1
    else:
        net_t += o_t_L - truth_1[t] # at time stemp t+1
        w_t   += o_t - o_t_L # at time stemp t+1
    ip_t  += o_t - truth_1[t] # at time stemp t+1
    return ip_t, net_t, w_t

## Costs Func.

In [7]:
def cost(
    h = 1,  # holding costs
    b = 19, # backlog costs
    net_invt = 0):
    ch = h*max(0, net_invt)
    cb = b*max(0,-net_invt)
    return ch, cb

## FF

In [10]:
import pandas as pd
def single_tsis(
    j:int, # the jth time series
    fcst, # the forecasted demand
    truth,# the true values
    lead_time = 3, # lead time
    h = 1,
    b = 19,
    a:float = 0.90,
    # initial position
    ip_t = 0 ,# initial inventory
    wig  = 0 ,# work-in-progress
    net = 0 ,# initial net inventory
    period:int = 28, 
):
    #
    DtL_l, ot_l, sst_l, ch_l, cb_l, ipt_l, net_l, wig_l = [], [], [], [], [], [], [], []
    #
    fcst_1 = np.array(fcst)[j:(j+1)*period]
    truth_1= np.array(truth)[j:(j+1)*period]
    errors = np.array(fcst_1 - truth_1)
    # ot
    for i in range(period):
        if i < 3: # t0
            DtL = ob_D_tL(t = i, fcst_1 = fcst_1, L=lead_time)
            DtL_l.append(DtL)
            
            ss_t = ob_ss_t(fcst_1_errors = errors, alpha = a, L=lead_time)
            
            o_t =  DtL + ss_t - ip_t # order simulation
            ot_l.append(o_t)
            
            ch,cb = cost(h=1,b=19, net_invt = net)
            ch_l.append(ch)
            cb_l.append(cb)
            
            ip_t, net, wig = ob_ip_t(ip_t = ip_t, net_t = net, w_t = wig, o_t = o_t, truth_1=truth_1, fcst_1= fcst_1, t = i , o_t_L = 0) ################
            ipt_l.append(ip_t)
            net_l.append(net)
            wig_l.append(wig)
            
        else:
            DtL = ob_D_tL(t = i, fcst_1 = fcst_1, L=lead_time)
            DtL_l.append(DtL)
            
            ss_t = ob_ss_t(fcst_1_errors = errors, alpha = a, L=lead_time)
            
            o_t = max(0, DtL + ss_t - ip_t) # order simulation
            ot_l.append(o_t)
            
            ch,cb = cost(h=1,b=19, net_invt = net)
            ch_l.append(ch)
            cb_l.append(cb)
            
            ip_t, net, wig = ob_ip_t(ip_t = ip_t, net_t = net, w_t = wig, o_t = o_t, truth_1=truth_1, fcst_1= fcst_1, t = i, o_t_L = ot_l[i-lead_time])
            ipt_l.append(ip_t)
            net_l.append(net)
            wig_l.append(wig)
    return pd.DataFrame({"DtL":DtL_l, "ot":ot_l,"ipt":ipt_l,"net":net_l, "wig": wig_l, "ch": ch_l, "cb":cb_l})

## Simulation Results:

In [12]:
np.random.seed(42)
n_p = 28 # the period
fcst = np.random.poisson(lam=55, size=n_p) # two sampling "time series" from Poisson distribution
truth= np.random.poisson(lam=50, size=n_p) 
#ds = single_tsis(j=42840, fcst = fcst['glgbm'], truth = truth['sarima'])
ds = single_tsis(j=0, fcst = fcst, truth = truth)
ds

,DtL,ot,ipt,net,wig,ch,cb
0,158,177.67437,123.67437,-1.333333,125.007703,0.00000,0.000000
1,163,59.00000,126.67437,-4.666667,131.341037,0.00000,25.333333
2,166,59.00000,138.67437,1.000000,137.674370,0.00000,88.666667
3,167,48.00000,127.67437,119.674370,8.000000,1.00000,0.000000
4,161,53.00000,134.67437,132.674370,2.000000,119.67437,0.000000
5,151,36.00000,124.67437,145.674370,-21.000000,132.67437,0.000000
6,161,56.00000,125.67437,138.674370,-13.000000,145.67437,0.000000
7,160,54.00000,119.67437,131.674370,-12.000000,138.67437,0.000000
8,161,61.00000,140.67437,127.674370,13.000000,131.67437,0.000000
9,152,31.00000,115.67437,127.674370,-12.000000,127.67437,0.000000


## Loop for future 42840 time series.

In [ ]:
from tqdm import tqdm
for j in tqdm(range(42840)):
    lis = []
    ds = single_tsis(j=42840, fcst = fcst['glgbm'], truth = truth['sarima'])
    lis.append(ds)

In [32]:
pd.DataFrame(ds)

,DtL,ot,ipt,net,wig,ch,cb
0,7.216594,13.240463,11.028264,0.193333,10.834931,0.000000,0
1,6.733274,1.728878,10.604712,0.446434,10.158278,0.193333,0
2,6.857690,2.276847,10.730777,0.701184,10.029593,0.446434,0
3,7.679585,2.972676,11.554320,11.792514,-0.238194,0.701184,0
4,8.393541,2.863089,12.269925,11.373908,0.896017,11.792514,0
5,8.791954,2.545897,12.669986,11.504919,1.165068,11.373908,0
6,8.610709,1.964591,12.490390,12.333407,0.156983,11.504919,0
7,7.947946,1.481424,11.829275,13.053958,-1.224682,12.333407,0
8,7.204127,1.398720,11.087106,13.458965,-2.371859,13.053958,0
9,7.055933,1.992695,10.940560,13.284314,-2.343754,13.458965,0


In [27]:
a = truth['sarima'][-28:].values
a

array([1.00943258, 1.1282362 , 1.14268638, 1.14490965, 1.14570032,
       1.14632313, 1.14692627, 1.14752711, 1.14812768, 1.14872821,
       1.14932875, 1.14992928, 1.15052981, 1.15113034, 1.15173088,
       1.15233141, 1.15293194, 1.15353247, 1.154133  , 1.15473354,
       1.15533407, 1.1559346 , 1.15653513, 1.15713567, 1.1577362 ,
       1.15833673, 1.15893726, 1.15953779])

In [28]:
b = fcst['glgbm'][-28:].values
b

array([0.18891868, 0.16439735, 0.14893482, 0.13364309, 0.12444357,
       0.11354703, 0.87091641, 1.42778727, 1.36482665, 1.61767561,
       1.08154828, 1.17855172, 1.41001135, 1.67917105, 1.67814852,
       1.58632277, 1.25666149, 0.97854054, 0.76554697, 1.08445969,
       1.42449112, 1.09097022, 1.41454696, 1.8320163 , 1.22411907,
       1.55662141, 1.68120609, 2.23894342])

In [29]:
sum(a)

32.08839639427592

In [30]:
sum(b)

31.316967418007778